In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration


# Parameters to be defined
model_checkpoint = "google/flan-t5-small"

tokenizer: T5Tokenizer = T5Tokenizer.from_pretrained(model_checkpoint, model_max_length=512)
model: T5ForConditionalGeneration = T5ForConditionalGeneration.from_pretrained(model_checkpoint)

In [2]:
input_ids = tokenizer("The <extra_id_0> walks in <extra_id_1> park", return_tensors="pt").input_ids
labels = tokenizer("<extra_id_0> cute dog <extra_id_1> the <extra_id_2>", return_tensors="pt").input_ids

decoded_input = tokenizer.decode(input_ids[0])
decoded_labels = tokenizer.decode(labels[0])

print(decoded_input)
print(decoded_labels)

# the forward function automatically creates the correct decoder_input_ids
loss = model(input_ids=input_ids, labels=labels).loss
loss.item()

The<extra_id_0> walks in<extra_id_1> park</s>
<extra_id_0> cute dog<extra_id_1> the<extra_id_2></s>


19.586868286132812

In [3]:
out = model.generate(input_ids, max_length=100, num_beams=5, early_stopping=True)

tokenizer.decode(out[0])

'<pad> The dog is walking in the park.</s>'

In [4]:
from datasets import load_dataset

# Load dataset
datasets = load_dataset("esnli")

Found cached dataset esnli (C:/Users/Remco/.cache/huggingface/datasets/esnli/plain_text/0.0.2/a160e6a02bbb8d828c738918dafec4e7d298782c334b5109af632fec6d779bbc)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
# the following 2 hyperparameters are task-specific
max_source_length = 512
max_target_length = 128

# Suppose we have the following 2 training examples:
input_sequence_1 = "Welcome to NYC"
output_sequence_1 = "Bienvenue à NYC"

input_sequence_2 = "HuggingFace is a company"
output_sequence_2 = "HuggingFace est une entreprise"

# encode the inputs
task_prefix = "translate English to French: "
input_sequences = [input_sequence_1, input_sequence_2]

encoding = tokenizer(
    [task_prefix + sequence for sequence in input_sequences],
    padding="longest",
    max_length=max_source_length,
    truncation=True,
    return_tensors="pt",
)

input_ids, attention_mask = encoding.input_ids, encoding.attention_mask

# encode the targets
target_encoding = tokenizer(
    [output_sequence_1, output_sequence_2],
    padding="longest",
    max_length=max_target_length,
    truncation=True,
    return_tensors="pt",
)
labels = target_encoding.input_ids

# replace padding token id's of the labels by -100 so it's ignored by the loss
labels[labels == tokenizer.pad_token_id] = -100

# forward pass
# No need to give right shift token ids as the model automatically creates them.
# And tokenizer add EOS token at the end of the sequence
loss = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels).loss
loss.item()

0.7163270711898804

In [6]:
def get_preprocess_function(tokenizer, training=True, max_source_length=512, max_target_length=128):
    def _preprocess_fn(examples):
        input_text = ['premise: ' + premise + ' \n ' + 'hypothesis: ' + hypothesis
                      for premise, hypothesis in zip(examples['premise'], examples['hypothesis'])]

        model_inputs = tokenizer(input_text, truncation=True, max_length=max_source_length)

        if training:
            target_text = examples['explanation_1']
            targets = tokenizer(target_text, truncation=True, max_length=max_target_length)

            model_inputs["labels"] = targets["input_ids"]
        else:
            for i in range(1, 4):
                key_explanation = 'explanation_' + str(i)
                target_text = examples[key_explanation]
                targets = tokenizer(target_text, truncation=True, max_length=max_target_length)
                model_inputs[key_explanation] = targets["input_ids"]


        return model_inputs
    return _preprocess_fn

In [10]:
raw_dataset_columns = datasets['train'].column_names

preprocess_function_train = get_preprocess_function(tokenizer, training=True)
datasets['train'] = datasets['train'].map(preprocess_function_train, batched=True, remove_columns=raw_dataset_columns)

preprocess_function_test = get_preprocess_function(tokenizer, training=False)
datasets['validation'] = datasets['validation'].map(preprocess_function_test, batched=True, remove_columns=raw_dataset_columns)
datasets['test'] = datasets['test'].map(preprocess_function_test, batched=True, remove_columns=raw_dataset_columns)

Loading cached processed dataset at C:\Users\Remco\.cache\huggingface\datasets\esnli\plain_text\0.0.2\a160e6a02bbb8d828c738918dafec4e7d298782c334b5109af632fec6d779bbc\cache-d67e13d575ddf396.arrow
Loading cached processed dataset at C:\Users\Remco\.cache\huggingface\datasets\esnli\plain_text\0.0.2\a160e6a02bbb8d828c738918dafec4e7d298782c334b5109af632fec6d779bbc\cache-eb5e9201f9aa7afa.arrow


In [11]:
datasets['validation'][0]

print(tokenizer.decode(datasets['train'][0]['input_ids']))
print(tokenizer.decode(datasets['train'][0]['labels']))

{'explanation_1': [37, 12, 281, 6307, 164, 59, 36, 45, 3074, 5, 1],
 'explanation_2': [1142,
  250,
  192,
  887,
  33,
  3,
  24209,
  6,
  405,
  59,
  1243,
  79,
  33,
  14033,
  5,
  2759,
  887,
  24,
  33,
  3,
  24209,
  33,
  59,
  6539,
  18233,
  3896,
  23281,
  5,
  1],
 'explanation_3': [2759,
  887,
  103,
  59,
  43,
  12,
  36,
  14033,
  5,
  3,
  17467,
  3738,
  53,
  405,
  59,
  1243,
  18233,
  3896,
  23281,
  5,
  37,
  887,
  103,
  59,
  43,
  12,
  43,
  131,
  2369,
  3074,
  12,
  36,
  3,
  24209,
  5,
  1],
 'input_ids': [3,
  17398,
  10,
  2759,
  887,
  33,
  3,
  24209,
  298,
  3609,
  12,
  281,
  6307,
  5,
  22455,
  10,
  37,
  14033,
  33,
  18233,
  3896,
  23281,
  298,
  3609,
  12,
  281,
  6307,
  227,
  131,
  3182,
  3074,
  5,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [9]:
from transformers import DataCollatorForSeq2Seq

# Use data collator to create batched data. This will pad the inputs and labels to the maximum length of the batch.
# Might be more efficient to pad to fixed length.
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True, label_pad_token_id=-100)



In [11]:
pass

In [12]:
from torch.utils.data import DataLoader

# Create dataloader
train_dataloader = DataLoader(datasets['train'], shuffle=True, batch_size=16, collate_fn=data_collator)

# Now we can iterate over the dataloader to get batches of data
data = next(iter(train_dataloader))
data['input_ids'].shape, data['attention_mask'].shape, data['labels'].shape

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ c:\Users\Remco\PythonEnvs\nlpenv\lib\site-packages\transformers\tokenization_utils_base.p │
│ y:717 in convert_to_tensors                                                               │
│                                                                                           │
│    714 │   │   │   │   │   value = [value]                                                │
│    715 │   │   │   │                                                                      │
│    716 │   │   │   │   if not is_tensor(value):                                           │
│ ❱  717 │   │   │   │   │   tensor = as_tensor(value)                                      │
│    718 │   │   │   │   │                                                                  │
│    719 │   │   │   │   │   # Removing this for now in favor of controlling the shape with │
│    720 │   │   │   │   │   # # at-least2d                                                 │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: too many dimensions 'str'

During handling of the above exception, another exception occurred:

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ C:\Users\Remco\AppData\Local\Temp\ipykernel_19948\1586504221.py:7 in <module>             │
│                                                                                           │
│   4 train_dataloader = DataLoader(preprocessed_datasets['train'], shuffle=True, batch_siz │
│   5                                                                                       │
│   6 # Now we can iterate over the dataloader to get batches of data                       │
│ ❱ 7 data = next(iter(train_dataloader))                                                   │
│   8 data['input_ids'].shape, data['attention_mask'].shape, data['labels'].shape           │
│ c:\Users\Remco\PythonEnvs\nlpenv\lib\site-packages\torch\utils\data\dataloader.py:628 in  │
│ __next__                                                                                  │
│                                                                                           │
│    625 │   │   │   if self._sampler_iter is None:                                         │
│    626 │   │   │   │   # TODO(https://github.com/pytorch/pytorch/issues/76750)            │
│    627 │   │   │   │   self._reset()  # type: ignore[call-arg]                            │
│ ❱  628 │   │   │   data = self._next_data()                                               │
│    629 │   │   │   self._num_yielded += 1                                                 │
│    630 │   │   │   if self._dataset_kind == _DatasetKind.Iterable and \                   │
│    631 │   │   │   │   │   self._IterableDataset_len_called is not None and \             │
│                                                                                           │
│ c:\Users\Remco\PythonEnvs\nlpenv\lib\site-packages\torch\utils\data\dataloader.py:671 in  │
│ _next_data                                                                                │
│                                                                                           │
│    668 │                                                                                  │
│    669 │   def _next_data(self):                                                          │
│    670 │   │   index = self._next_index()  # may raise StopIteration                      │
│ ❱  671 │   │   data = self._dataset_fetcher.fetch(index)  # may raise StopIteration       │
│    672 │   │   if self._pin_memory:                                                       │
│    673 │   │   │   data = _utils.pin